# Containers

Optional resources, read after finishing the course if you want more details:

1. __[Containers explained by google](https://cloud.google.com/learn/what-are-containers#:~:text=Containers%20are%20packages%20of%20software,on%20a%20developer's%20personal%20laptop.)__
2. __[Podman vs. Docker](https://phoenixnap.com/kb/podman-vs-docker)__
3. __[Optimizing the size of images](https://devopscube.com/reduce-docker-image-size/)__

## What are they?
Containers are packages of software that contain all of the necessary elements to run in any environment. In this way, containers virtualize the operating system and run anywhere, from a private data center to the public cloud or even on a developer’s personal laptop. You only need to correctly setup a container manager such as Docker or Podman.

## Why do we use them?
The most important benefits of containers are:
+ **Separation of responsibility** - Developers focus on application logic and dependencies while IT operations engineers focus on deployment instead of managing software versions and configurations.
+ **Workload portability** - They can run everywhere (on most operating systems; on virtual machines or physical servers; on your personal computer or a data center).
+ **Application isolation** - Containers virtualize CPU, memory, storage and network resources at the operating system level, providing a view of the OS logically isolated from other containers. Communication between containers needs to be explicitly configured.

## Containers vs. Virtual Machines
Virtual machines can do many of the same things, but containers are faster and more lightweight. Containers virtualize at the operating system level while VMs virtualize at the hardware level. This means that containers share the kernel of the host operating system, use a lot less memory and are faster than VMs. Because of this a container whose image is based on the Linux kernel can only run on Linux (or Windows with the subsystem for Linux enabled).

## Podman vs. Docker
For this tutorial we'll use docker but the industry is moving away from it in favor of podman. Podman was built to seamlessly replace Docker so its commands are mostly the same. The main differences are:

1. **Architecture** - Docker has a client-server architecture. The client is the CLI app (the commands you run) and it communicates with the server (the docker daemon) through a REST API. The containers are thus children of the daemon process. Podman is decentralized in the sense that every time you run a new container it doesn't have to go through a single point of failure (the daemon) and it creates a new podman process which forks into the container child process.
2. **Root Privileges** - By default Docker requires root privileges to communicate with the daemon and the process of making it run rootless is tedious. Podman is rootless by design.
3. **Security** - While Docker is considered to be a secure tool when users apply the security best practices for it, podman is considered by most to be more secure.
4. **Other Tools** - Only docker supports docker swarm, but podman users can use Nomad for example. Docker compose is supported by both of them as of podman version 3.0.

In [1]:
# here are some helper functions which you can mostly ignore
# but make sure you run this cell
remove_containers () {
    # removes all containers that use a given image
    # $1 - name of the image for which to remove all containers

    # find all containers that share a given image as an ancestor
    CONTAINERS="$(docker ps -q -a --filter ancestor=$1)"
    # if there is at least one such container
    if [ ! -z "$CONTAINERS" ]; then
        # stop and remove those containers
        docker rm $(docker stop $CONTAINERS) &> /dev/null
    fi
}

remove_image () {
    # remove an image given its name and remove all containers
    # that depend on it
    # $1 - name of the image to remove

    remove_containers $1

    # remove the given image now that we're sure no containers
    # depend on it
    docker rmi $1 &> /dev/null
}

## Getting started with Docker

Let's first run the Hello World container and see what we can learn.

In [2]:
# you can ignore this line; it's only here to make sure every time
# you run the next command you get the same output
remove_image hello-world:latest

# this is the command whose output we want to inspect
docker run hello-world

Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world

Digest: sha256:80f31da1ac7b312ba29d65080fddf797dd76acfb870e677f390d5acba9741b17
Status: Downloaded newer image for hello-world:latest

Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https

### Images and Registries
What we did here was run a docker image called "hello-world". An image is pre-built environment that starts from a base operating system (for example, ubuntu 18.04) on which have been installed all dependencies an app needs. One way to think about images is through the lens of object oriented programming. An <span style="color:#00cc00">**image**</span> can be thought of as a <span style="color:#00cc00">**class**</span> and the <span style="color:#ff66ff">**instances**</span> of that class are the <span style="color:#ff66ff">**containers**</span>. This "hello-world" image was not on your computer when you installed Docker.
```bash
Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world
```
Hence why it's telling you it cannot find it and that it's trying to pull (download) it from the **registry**. A **registry** hosts and serves to clients whatever images they need. The default registry with docker is ___[Docker Hub](https://hub.docker.com/)___ where you can upload your own images for free.

You can think of Docker Hub as a GitHub for container images. However, on GitHub you usually find only application source code, while the container image usually contains the compiled application along with the environment necessary for running it (base operating system, configuration files, libraries, etc.).

### Docker Steps
The "hello-world" container has outputed a message detailing what steps were followed in running it:

```bash
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.
```

As we discussed, docker uses a client-server architecture and the daemon does the actual work of downloading the image and running the container (spawning a new process based on the image). The client only transmits the command and prints the output to the terminal.

## More about Images

In [3]:
docker image list

REPOSITORY                       TAG       IMAGE ID       CREATED        SIZE
nginx-static                     latest    e832471c85be   4 hours ago    23.4MB
cosminc98/4psa-version-control   latest    70d10440db69   30 hours ago   1.41GB
hello-world                      latest    feb5d9fea6a5   8 months ago   13.3kB


The above command's output probably looks like:
```bash
REPOSITORY                       TAG       IMAGE ID       CREATED        SIZE
cosminc98/4psa-version-control   latest    70d10440db69   4 hours ago    1.41GB
hello-world                      latest    feb5d9fea6a5   8 months ago   13.3kB
```
In the REPOSITORY column you see the image's name. It's composed of the user's name (cosminc98) and the project's name. The entries in the TAG column can be thought of as different versions of the same project. Obviously the "latest" tag means that this is the most recent version. You can also see how big an image is. There are many techniques to reduce image sizes which are beyond the scope of this guide so we'll discuss only one.

### Creating a new image with Dockerfile

A Dockerfile is a configuration file from which we can create a container image. Each instruction creates a new layer which is an image in itself which can be cached to save time if you need to rebuild later. A layer stores changes compared to the layer it was based on. Because of this you can reduce image sizes by deleting temporary files or undoing changes before the end of an instruction. This can be combined with chaining multiple commands to create as few layers as possible.

```bash
RUN apt-get update && apt-get install -y\
         [package-one] \
         [package-two] 
   && rm -rf /var/lib/apt/lists/*
```

In [ ]:
cat Dockerfile

: 

We'll create a container that will use Nginx to host a static web page in a container and see it in a browser on your host operating system.

1. The first instruction "<span style="color:#cc6600">FROM</span> nginx:mainline-alpine" sets the base image from which we'll make modifications. It first tries to find the image locally and if it's not found it downloads it from a registry (probably Docker Hub because it is the default for docker).
2. "<span style="color:#cc6600">RUN</span>  rm /etc/nginx/conf.d/*" removes some configuration files. Another example would be installing a python package: "<span style="color:#cc6600">RUN</span> pip install tqdm"
3. "<span style="color:#cc6600">ADD</span>  hello.conf /etc/nginx/conf.d/"; the add command copies files/directories into a Docker image.
4. "<span style="color:#cc6600">ADD</span>  index.html /usr/share/nginx/html/" adds the actual content of the web page. You can modify this page for your amusement.

___[Here](https://kapeli.com/cheat_sheets/Dockerfile.docset/Contents/Resources/Documents/index)___ is a cheat sheet describing all Dockerfile commands in depth.


In [5]:
# ignore
remove_image nginx-static:latest

# here we create an image with the name "nginx-static" and the version "latest"
docker build -t nginx-static:latest .

[+] Building 0.0s (0/1)                                                         
[+] Building 0.2s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 246B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/nginx:mainline-alpine   0.1s
[+] Building 0.3s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 246B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for dock

In [6]:
# the image has been added to docker's database
docker image list

REPOSITORY                       TAG       IMAGE ID       CREATED        SIZE
nginx-static                     latest    e98e248b550f   4 hours ago    23.4MB
cosminc98/4psa-version-control   latest    70d10440db69   30 hours ago   1.41GB
hello-world                      latest    feb5d9fea6a5   8 months ago   13.3kB


## docker run

This command starts a container using an image. There are many options that can be used. Among the most important:
+ "-p 1234:4321" maps the port 1234 from the **host** to the port 4321 on the **container**.
+ "-d" option runs the container in detached mode (background), meaning you can continue to use the terminal
+ "-v /path/in/host:/path/in/container" mounts a directory / files from the host's file system to the container's file system. This option you already used twice to start this container. The first time was '-v "$(pwd)":/usr/project' which added all files in the current directory (the notebooks, Dockerfiles, etc.) to "/usr/project". The second time was "-v /var/run/docker.sock:/var/run/docker.sock" which mounted the unix socket that the docker daemon was listening on the host. This means that whenever we run a docker command it has the same effect as if we ran it on the host (outside all containers).
+ "--rm" when a container exits after doing its job its file system will be automatically cleaned. Everything is lost except the files that were mounted with the "-v" option. This is why you think of containers as temporary entities whose only persistent components are volumes (mounted files and directories).


In [7]:
# ignore
remove_containers nginx-static:latest

# start the web server
docker run -p 8081:80 -d nginx-static

f6f0bbeec3a24d4c264fd5a292394ecc2db6c27f74b4f6b4c1d83d6b871d2da6


The command above starts the web server (a container with that server). 
+ "-p 8081:80" maps port 8081 from the host to container's port 80 on which the web server is listening.
+ We can now open the browser on http://localhost:8081 and see the website.

## docker ps
This command lists all containers. There are many options that can be used. Among the most important:
+ '-a' shows both active containers and those who have exited
+ '--filter ancestor=image-name:version' filters out the containers that do not use a specific version of an image
+ '-q' only display the containers' ids

In [8]:
docker ps -a --filter ancestor=nginx-static:latest

CONTAINER ID   IMAGE          COMMAND                  CREATED         STATUS         PORTS                  NAMES
f6f0bbeec3a2   nginx-static   "/docker-entrypoint.…"   6 seconds ago   Up 6 seconds   0.0.0.0:8081->80/tcp   beautiful_lamarr


## docker stop & docker start & docker rm

Remember that stopping a container does not initiate the cleanup sequence for its file system and can be restarted with no problems.

In [9]:
# get the id of the web server container; an alternative to using ids
# is naming containers and using that insted of the ids
CONTAINER_ID=$(docker ps -a --filter ancestor=nginx-static:latest -q)

# after running this command you can check in your browser that the
# web server is down
docker stop $CONTAINER_ID

f6f0bbeec3a2


In [10]:
# and now it's gonna be up again
docker start $CONTAINER_ID

f6f0bbeec3a2


In [11]:
# in this guide we've already been removing containers and images
# so you can test these commands yourself; hints you can only remove
# containers you've stopped and you can only remove images for
# which there are no more containers (stopped or otherwise)

# docker rm $CONTAINER_ID
# docker rmi nginx-static:latest

# if you do run this, make sure to rebuild the nginx-static image
# and create the web server again

## docker exec
This runs a command inside an existing container.

In [12]:
# if you inspect the html tags on http://localhost:8081 you'll 
# see they're the exact same because that's the file on the
# container being server by the web server

docker exec $CONTAINER_ID "cat" "/usr/share/nginx/html/index.html" | head -n 4

<!DOCTYPE html>
<html>
<head>
<title>Hello World</title>


In [13]:
# these commands you need to run in the terminal
# CONTAINER_ID=$(docker ps -a --filter ancestor=nginx-static:latest -q)
# docker exec -it $CONTAINER_ID /bin/bash

# now that you have a shell running in the container
# cat /usr/share/nginx/html/index.html | head -n 4

## docker logs

Docker also has a logging mechanism to keep track of what's happened inside a container. You can learn more about it __[here](https://docs.docker.com/engine/reference/commandline/logs/)__. We'll see a quick example.

In [14]:
# take note that you'll have to manually stop this cell from
# executing since it's waiting for more logs to appear
docker logs --follow $CONTAINER_ID

# at the end of the output you'll see new lines every time you hit
# refresh on your browser on http://localhost:8081 

/docker-entrypoint.sh: /docker-entrypoint.d/ is not empty, will attempt to perform configuration
/docker-entrypoint.sh: Looking for shell scripts in /docker-entrypoint.d/
/docker-entrypoint.sh: Launching /docker-entrypoint.d/10-listen-on-ipv6-by-default.sh
10-listen-on-ipv6-by-default.sh: info: /etc/nginx/conf.d/default.conf is not a file or does not exist
/docker-entrypoint.sh: Launching /docker-entrypoint.d/20-envsubst-on-templates.sh
/docker-entrypoint.sh: Launching /docker-entrypoint.d/30-tune-worker-processes.sh
/docker-entrypoint.sh: Configuration complete; ready for start up
2022/05/26 18:26:53 [notice] 1#1: using the "epoll" event method
2022/05/26 18:26:53 [notice] 1#1: nginx/1.21.6
2022/05/26 18:26:53 [notice] 1#1: built by gcc 10.3.1 20211027 (Alpine 10.3.1_git20211027) 
2022/05/26 18:26:53 [notice] 1#1: OS: Linux 5.10.16.3-microsoft-standard-WSL2
2022/05/26 18:26:53 [notice] 1#1: getrlimit(RLIMIT_NOFILE): 1048576:1048576
2022/05/26 18:26:53 [notice] 1#1: start worker proces

# Assignment

You may have noticed that the static website's image is missing. Your task is to fix this, document the steps you've taken and answer some questions.
+ First look into "index.html" and find where the image tag source path. Also you need to read __[here](https://docs.nginx.com/nginx/admin-guide/web-server/serving-static-content/)__ how static serving works and have a look in "hello.conf".
+ After you discover the container path where you need to put the image you have to actually put it there. There are at least two ways to do this and you can chose either one to solve the problem. It is highly recommended to solve this assignment on your host operating system, not in the container we set up for the notebook server.
+ Answer why one method is preferable over the other (what are the advantages).

Solution:

Cele doua metode pe care le-as putea folosi pentru a adauga logo-ul in imaginea container-ului sunt fie folosind instructiunea ADD in Dockerfile inainte de a buildui imaginea container-ului, fie cu comanda docker cp in terminal cat timp containerul ruleaza. Varianta cu docker cp nu este o varianta ideala pentru automatizare sau cazuri in care sunt necesare actualizari frecvente, asa ca am ales sa folosesc varianta cu ADD, fiind o modalitate care nu necesita interventie manuala pentru a copia imaginea in container.